# **Smart Statement Reader**

> **An AI/ML-powered solution for processing financial PDFs with precision and efficiency.**  

#### Detect PDF Type (Structured vs. Scanned)

In [1]:
import pdfplumber

def is_scanned_pdf(pdf_path):
    """Check if the PDF is scanned or structured"""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text and len(text.strip()) > 10:
                return False  # Text-based PDF
    return True  # Scanned PDF

#### _Testing_

In [2]:
pdf_path = "Sample1.pdf"
print("Scanned PDF:", is_scanned_pdf(pdf_path))

Scanned PDF: False


#### Extract Text Based on PDF Type

##### For Text-Based PDFs (Structured PDFs)

In [3]:
def extract_text_from_pdf(pdf_path):
    """Extract text from structured PDFs using pdfplumber"""
    text_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text_data.append(page.extract_text())
    return "\n".join(text_data)

##### For Scanned PDFs (OCR-Based Extraction)

In [4]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import pytesseract

def extract_text_from_scanned_pdf(pdf_path):
    """Extract text from scanned PDFs using OCR"""
    text_data = []
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        img = doc[page_num].get_pixmap()  # Convert to image
        img_array = np.frombuffer(img.samples, dtype=np.uint8).reshape(img.height, img.width, 3)
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        text = pytesseract.image_to_string(gray)
        text_data.append(text)
    
    return "\n".join(text_data)

#### _Testing_

In [5]:
scanned = is_scanned_pdf(pdf_path)
if scanned:
    text = extract_text_from_scanned_pdf(pdf_path)
else:
    text = extract_text_from_pdf(pdf_path)

print(text)  # Print first 1000 characters for verification

Bank name
COMPANY NAME AND Address 1
Address 2
LOGO Address 3
SWIFT :
E-mail :
Website :
CUSTOMER NAME
ADDRESS 1
ADDRESS 2
ADDRESS 3
ADDRESS 4
ADDRESS 5
Ref: INB 0000-30001- Customer Id : 01111113
000000 Account No : 4304567890987
Account Type : CURRENT ACCOUNT
Currency : BDT
Issue Date : January 12, 2019
STATEMENT OF ACCOUNT FOR THE PERIOD OF 29-Dec-2018 TO 12-Jan-2019
DATE PARTICULARS CHQ.NO WITHDRAW DEPOSIT BALANCE
Balance Forward 0.00 0.00 419.08
01-Jan-2019 PRCR/easy 100.00 0.00 319.08
D/4301201810853001
01-Jan-2019 PRCR/easy 100.00 0.00 219.08
D/4301201810853001
01-Jan-2019 PRCR/easy 100.00 0.00 119.08
D/4301201810853001
01-Jan-2019 PRCR/easy 100.00 0.00 19.08
D/4301201810853001
02-Jan-2019 EFT/IC/BANK ASIA/FRD BEFTN 0.00 10,103.66 10,122.74
ACCOUN/FR
02-Jan-2019 PRCR/easy 100.00 0.00 10,022.74
D/4301201810853001
02-Jan-2019 PRCR/easy 100.00 0.00 9,922.74
D/4301201810853001
03-Jan-2019 CWDR/MOKKA-MODINA TOWER CT 9,500.00 0.00 422.74
G/4301201810853001
07-Jan-2019 EFT/IC/BANK ASIA

#### Extract Financial Data (Ledger Entries)

In [6]:
import re

def preprocess_text(text):
    # Step 1: Remove excessive newlines, keeping structured breaks
    text = re.sub(r"\n([A-Za-z/]+)", r" \1", text)  # Merge split lines for transaction descriptions
    text = re.sub(r"(?<=\d{4})\n(?=\d{2})", " ", text)  # Fix split dates like "2019\n01"

    # Step 2: Ensure proper column spacing
    text = re.sub(r"\s{2,}", " ", text)  # Replace multiple spaces with a single space

    return text


In [7]:
def extract_ledger_entries(text):
    # Preprocess text first
    text = preprocess_text(text)

    # Updated regex to match transactions properly
    pattern = re.compile(
        r'(\d{1,2}[-/.][A-Za-z]{3}[-/.]\d{4})'  # Match Date (e.g., 01-Jan-2019, 02/01/2019)
        r'\s+([\w\s/&.,-]+?)'                    # Match Description (e.g., PRCR/easy, EFT)
        r'\s*(-?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?'  # Match Withdraw (optional)
        r'\s*(-?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?'  # Match Deposit (optional)
        r'\s*(-?\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'   # Match Balance
    )

    matches = pattern.findall(text)

    # Debugging: Print first few matches
    print("Found matches:", matches[:5])  

    if not matches:
        print("No matches found! Check text formatting.")
        return []

    transactions = []
    for match in matches:
        date, description, withdraw, deposit, balance = match

        transactions.append({
            "Date": date.strip(),
            "Description": description.strip(),
            "Withdraw": float(withdraw.replace(",", "")) if withdraw else 0.0,
            "Deposit": float(deposit.replace(",", "")) if deposit else 0.0,
            "Balance": float(balance.replace(",", "")) if balance else 0.0
        })

    return transactions


#### _Testing_

In [8]:
transactions = extract_ledger_entries(text)

if transactions:
    print("Extracted Transactions:", transactions[:5])  # Show first 5 transactions
else:
    print("Still no matches found! Check extracted text formatting.")

Found matches: [('29-Dec-2018', 'TO', '1', '', '2'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '319.08'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '219.08'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '119.08'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '19.08')]
Extracted Transactions: [{'Date': '29-Dec-2018', 'Description': 'TO', 'Withdraw': 1.0, 'Deposit': 0.0, 'Balance': 2.0}, {'Date': '01-Jan-2019', 'Description': 'PRCR/easy', 'Withdraw': 100.0, 'Deposit': 0.0, 'Balance': 319.08}, {'Date': '01-Jan-2019', 'Description': 'PRCR/easy', 'Withdraw': 100.0, 'Deposit': 0.0, 'Balance': 219.08}, {'Date': '01-Jan-2019', 'Description': 'PRCR/easy', 'Withdraw': 100.0, 'Deposit': 0.0, 'Balance': 119.08}, {'Date': '01-Jan-2019', 'Description': 'PRCR/easy', 'Withdraw': 100.0, 'Deposit': 0.0, 'Balance': 19.08}]


#### Save Data to CSV/Excel

In [9]:
import pandas as pd

def save_to_csv(transactions, output_file):
    """Save extracted data to CSV"""
    df = pd.DataFrame(transactions)
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

def save_to_excel(transactions, output_file):
    """Save extracted data to Excel"""
    df = pd.DataFrame(transactions)
    df.to_excel(output_file,index=False, engine='xlsxwriter')
    print(f"Data saved to {output_file}")

#### Combine Everything in One Function

In [12]:
def process_pdf(pdf_path, output_file):
    """Main function to process PDF and extract structured financial data"""
    scanned = is_scanned_pdf(pdf_path)

    if scanned:
        print("Processing scanned PDF with OCR...")
        extracted_text = extract_text_from_scanned_pdf(pdf_path)
    else:
        print("Processing structured PDF...")
        extracted_text = extract_text_from_pdf(pdf_path)

    transactions = extract_ledger_entries(extracted_text)

    if transactions:
        save_to_csv(transactions, output_file.replace(".xlsx", ".csv"))
        save_to_excel(transactions, output_file)
    else:
        print("No financial data found in the document.")

# Run the complete pipeline
pdf_file = "Sample1.pdf"  # Change this to your PDF file
output_file = "extracted_data.xlsx"
process_pdf(pdf_file, output_file)

Processing structured PDF...
Found matches: [('29-Dec-2018', 'TO', '1', '', '2'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '319.08'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '219.08'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '119.08'), ('01-Jan-2019', 'PRCR/easy', '100.00', '0.00', '19.08')]
Data saved to extracted_data.csv
Data saved to extracted_data.xlsx


In [13]:
def extract_format1(text):
    pattern = re.compile(
        r'(?P<date>\d{2}/\d{2}/\d{4})'    # Date (MM/DD/YYYY)
        r'\s+(?P<desc>[A-Za-z\s&]+?)'     # Description
        r'\s+(?P<debit>\d+\.\d{2})'       # Debit
        r'\s+(?P<credit>\d+\.\d{2})'      # Credit
    )
    transactions = [m.groupdict() for m in pattern.finditer(text)]
    return transactions

In [14]:
def extract_format2(text):
    pattern = re.compile(
        r'(?P<txn_id>[A-Z]{2}\d{5})'      # Transaction ID (e.g., AB12345)
        r'\s+\$(?P<amount>\d+,\d{3}\.\d{2})'  # Amount with $
        r'\s+(?P<date>\d{4}-\d{2}-\d{2})' # ISO date
    )
    transactions = [m.groupdict() for m in pattern.finditer(text)]
    return transactions

In [15]:
def extract_format3(text):
    pattern = re.compile(
        r'Date:\s*(?P<date>\d{2}-[A-Za-z]{3}-\d{4})\n'  # Date: 12-Jan-2023
        r'Desc:\s*(?P<desc>.*?)\n'                      # Description line
        r'Amt:\s*(?P<amount>[\d,]+\.\d{2})'             # Amount line
    )
    transactions = [m.groupdict() for m in pattern.finditer(text)]
    return transactions

In [16]:
def auto_detect_format(text):
    # Check for Format 1 markers
    if re.search(r'Debit\s+Credit', text):
        return extract_format1
    
    # Check for Format 2 markers
    if re.search(r'TXN ID', text):
        return extract_format2
    
    # Check for Format 3 markers
    if re.search(r'Date:\s*\d{2}-[A-Za-z]{3}-\d{4}', text):
        return extract_format3
    
    # Fallback to generic parser
    return extract_generic

def extract_generic(text):
    """Catch-all parser with optional fields"""
    pattern = re.compile(
        r'(?P<date>\d{2}[-/]\d{2}[-/]\d{4})'  # Date
        r'\s+(?P<desc>.+?)'                   # Description
        r'\s+(?P<amount>-?\$?\d{1,3}(?:,\d{3})*\.\d{2})'  # Amount
    )
    transactions = [m.groupdict() for m in pattern.finditer(text)]
    return transactions

In [17]:
test_text = "Sample header\n01/01/2023 Payment $100.00"
extractor = auto_detect_format(test_text)
extractor(test_text)

[{'date': '01/01/2023', 'desc': 'Payment', 'amount': '$100.00'}]

In [18]:
test_text1 = """Debit Credit
01/01/2023 Vendor Payment 1000.00 0.00
02/01/2023 Client Payment 0.00 2500.00"""
auto_detect_format(test_text1)(test_text1)

[{'date': '01/01/2023',
  'desc': 'Vendor Payment',
  'debit': '1000.00',
  'credit': '0.00'},
 {'date': '02/01/2023',
  'desc': 'Client Payment',
  'debit': '0.00',
  'credit': '2500.00'}]

In [19]:
test_text2 = """TXN ID Amount Date
AB12345 $1,234.56 2023-01-01
CD67890 $5,678.90 2023-01-02"""
auto_detect_format(test_text2)(test_text2)

[{'txn_id': 'AB12345', 'amount': '1,234.56', 'date': '2023-01-01'},
 {'txn_id': 'CD67890', 'amount': '5,678.90', 'date': '2023-01-02'}]